In [101]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound
from celmech.disturbing_function import laplace_b as b

In [102]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [103]:
def b1(alpha):
    return b(1.5, 1, 0, alpha)
def b2(alpha):
    return b(1.5, 2, 0, alpha)

In [104]:
def make_sim(alpha, mass, ecc, pom):
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)
    mu1, mu2, mu3 = mass
    ecc1, ecc2, ecc3 = ecc
    pomega1, pomega2, pomega3 = pom
    
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, e=ecc1, pomega=pomega1)
    sim.add(m=mu2, P=P2, e=ecc2, pomega=pomega2)
    sim.add(m=mu3, P=P3, e=ecc3, pomega=pomega3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    sim.dt = sim.particles[1].P/12
    sim.collision = "direct"
    
    return sim

In [248]:
def norun(sim):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    ec13 = 1-ps[1].a/ps[3].a
    m_tot = ps[1].m + ps[2].m + ps[3].m
    Tsec = 2*np.pi/(1/2*m_tot/ps[0].m/ec13**2)*ps[1].P
    
    Nout = 1000
    times = np.linspace(0,2*Tsec, Nout)
     
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    ec12, ec23, ec13 = 1-alpha12, 1-alpha23, 1-alpha13
    
    R2 = np.array([[np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)],
                   [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
                   [np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)]])
    
    massprefac = -1/(m1+m3)
    w2 = massprefac*(alpha23**(5/2)*m1*m2/ec23**2 + alpha12**(1/2)*m2*m3/ec12**2 + alpha12**(1/2)*alpha23**(5/2)*(m1+m3)**2/ec13**2)
    w1 = massprefac*(m_tot * (alpha23**(5/2)*m3/ec23**2 + alpha12**(1/2)*m1/ec12**2))
    k = massprefac * np.sqrt(m1*m2*m3*m_tot) * (alpha23**(5/2)*1/ec23**2 - alpha12**(1/2)*1/ec12**2)

#     kp = massprefac * np.sqrt(m1*m2*m3/m_tot) * (m2*(alpha23**(5/2)/ec23**2-alpha12**(1/2)/ec12**2) + (m1+m3)*(alpha23**(9/4)/ec23**2 - alpha12**(3/4)/ec12**2))
    delta = -np.pi/2*massprefac*np.sqrt(m1*m2*m3/m_tot) * (m3*alpha13*alpha23*alpha12**(-1/2)*b1(alpha13) - m1*alpha13*alpha23**(3/2)*b1(alpha13) + (m1-m3)*alpha13*alpha23**(5/4)*alpha12**(-1/4)*b2(alpha13))
    phi = 1/2*np.arctan2(2*k, w2-w1)
    R1 = np.array([[np.cos(phi), -np.sin(phi), 0],
                [np.sin(phi), np.cos(phi), 0],
                [0,0,1]])
    R = np.matmul(R1, R2)
    
    print('Mdiag elements:', Mdiag[0][0],Mdiag[1][1],Mdiag[2][2])
    Mdiagapprox = R2 @ lsys.Neccentricity_matrix @ R2.T
    Mp = R1 @ Mdiagapprox @ R1.T
    print('Mapprox elements:', Mp[0][0],Mp[1][1],Mp[2][2])
    
#     print('kp:', Mdiagapprox[1,0], kp+delta, (Mdiagapprox[1,0]-(kp+delta))/Mdiagapprox[1,0])
    print('-------------------------------------------------------------')
    print('w1:', Mdiagapprox[0,0], w1, (Mdiagapprox[0,0]-w1)/Mdiagapprox[0,0])
    print('w2:', Mdiagapprox[1,1], w2, (Mdiagapprox[1,1]-w2)/Mdiagapprox[1,1])
    print('w2-w1:', Mdiagapprox[1,1]-Mdiagapprox[0,0], w2-w1, ((Mdiagapprox[1,1]-Mdiagapprox[0,0])-(w2-w1))/(Mdiagapprox[1,1]-Mdiagapprox[0,0]))
    print('k:', Mdiagapprox[1,0], k, (Mdiagapprox[1,0]-k)/Mdiagapprox[1,0])
    
    tanphi = 2*Mdiagapprox[1,0]/(Mdiagapprox[1,1]-Mdiagapprox[0,0])
    print('tan phi:', tanphi, 2*k/(w2-w1), (tanphi-2*k/(w2-w1))/tanphi)

In [252]:
alpha = 0.85, 0.75 #0.735, 0.735
mass = 1e-7,1e-6,3e-7
ecc = 0.005, 0.005, 0.005
pomega = 0, 0, np.pi
sim = make_sim(alpha, mass, ecc, pomega)
norun(sim)

Mdiag elements: -4.8003868358066545e-05 -1.0876664475056001e-05 -9.64253294520452e-07
Mapprox elements: -4.797381355395936e-05 -1.0880813340427219e-05 -9.901592332564177e-07
-------------------------------------------------------------
w1: -2.300948471472688e-05 -2.2525390140561976e-05 0.021038914176772745
w2: -3.584514217965971e-05 -3.404736634253388e-05 0.05015396028045261
w2-w1: -1.283565746493283e-05 -1.1521976201971903e-05 0.1023462387142941
k: 1.740763066014729e-05 1.7000392985062325e-05 0.02339420470456593
tan phi: -2.712386289164407 -2.9509508936761786 -0.08795377172669055
